# CacheBackedEmbeddings로 임베딩 캐싱하기

이 튜토리얼에서는 LangChain의 `CacheBackedEmbeddings`를 사용하여 임베딩을 캐싱하는 방법을 알아봅니다. 임베딩 캐싱은 동일한 텍스트에 대한 임베딩을 재계산하지 않아 성능을 크게 향상시킬 수 있습니다.

## 목차
1. CacheBackedEmbeddings 소개
2. LocalFileStore를 사용한 영구 캐싱
3. InMemoryByteStore를 사용한 임시 캐싱
4. 성능 비교

## 1. CacheBackedEmbeddings 소개

Embeddings는 재계산을 피하기 위해 저장되거나 일시적으로 캐시될 수 있습니다.

Embeddings를 캐싱하는 것은 `CacheBackedEmbeddings`를 사용하여 수행될 수 있습니다. 캐시 지원 embedder는 embeddings를 키-값 저장소에 캐싱하는 embedder 주변에 래퍼입니다. 텍스트는 해시되고 이 해시는 캐시에서 키로 사용됩니다.

### 주요 매개변수

`CacheBackedEmbeddings`를 초기화하는 주요 지원 방법은 `from_bytes_store`입니다. 이는 다음 매개변수를 받습니다:

- **`underlying_embeddings`**: 임베딩을 위해 사용되는 embedder.
- **`document_embedding_cache`**: 문서 임베딩을 캐싱하기 위한 `ByteStore` 중 하나.
- **`namespace`**: (선택 사항, 기본값은 `""`) 문서 캐시를 위해 사용되는 네임스페이스. 이 네임스페이스는 다른 캐시와의 충돌을 피하기 위해 사용됩니다.

⚠️ **주의**: 동일한 텍스트가 다른 임베딩 모델을 사용하여 임베딩될 때 충돌을 피하기 위해 `namespace` 매개변수를 설정하는 것이 중요합니다.

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 2. LocalFileStore를 사용한 영구 캐싱

`LocalFileStore`는 임베딩을 로컬 파일 시스템에 영구적으로 저장합니다. 이는 프로그램이 종료되어도 캐시된 임베딩이 유지되어 다음 실행 시에도 사용할 수 있습니다.

먼저, 로컬 파일 시스템을 사용하여 임베딩을 저장하고 FAISS 벡터 스토어를 사용하여 검색하는 예제를 살펴보겠습니다.

In [2]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings()

# 로컬 파일 저장소 설정 (./cache/ 디렉토리에 저장)
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,  # 모델명을 네임스페이스로 사용하여 충돌 방지
)

### 캐시 저장소 확인

저장소에 어떤 키들이 있는지 확인할 수 있습니다:

In [3]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

['llm_cache.db']

### 문서 로드 및 처리

문서를 로드하고, 청크로 분할한 다음, 각 청크를 임베딩하고 벡터 저장소에 로드합니다.

In [4]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("../appendix-keywords.txt").load()

# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서 분할
documents = text_splitter.split_documents(raw_documents)

### 첫 번째 임베딩 생성 (캐시 없음)

처음 실행할 때는 캐시가 없으므로 모든 임베딩을 계산해야 합니다:

In [5]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: total: 406 ms
Wall time: 1.97 s


### 두 번째 임베딩 생성 (캐시 사용)

벡터 저장소를 다시 생성하려고 하면, 임베딩을 다시 계산할 필요가 없기 때문에 훨씬 더 빠르게 처리됩니다.

**성능 향상**: 약 30배 이상 빠른 처리 속도를 보여줍니다!

In [6]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 15.6 ms
Wall time: 65.7 ms


## 3. InMemoryByteStore를 사용한 임시 캐싱

`InMemoryByteStore`는 임베딩을 메모리에만 저장합니다. 프로그램이 종료되면 캐시가 사라지지만, 프로그램 실행 중에는 빠른 접근이 가능합니다.

다른 `ByteStore`를 사용하기 위해서는 `CacheBackedEmbeddings`를 생성할 때 해당 `ByteStore`를 사용하면 됩니다.

### 사용 사례
- 임시 작업이나 테스트 환경
- 디스크 I/O를 최소화하고 싶은 경우
- 짧은 세션 동안만 캐시가 필요한 경우

In [7]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

# 메모리 내 바이트 저장소 생성
store = InMemoryByteStore()

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, 
    store, 
    namespace=embedding.model
)

## 4. LocalFileStore vs InMemoryByteStore 비교

| 특성 | LocalFileStore | InMemoryByteStore |
|------|----------------|-------------------|
| **영속성** | 영구 저장 (디스크) | 임시 저장 (메모리) |
| **속도** | 중간 (디스크 I/O) | 매우 빠름 (메모리 접근) |
| **용량** | 디스크 용량에 의존 | RAM 크기에 제한 |
| **사용 사례** | 프로덕션 환경, 반복 작업 | 개발/테스트, 단기 세션 |
| **재시작 후** | 캐시 유지 | 캐시 손실 |

## 5. 모범 사례

1. **네임스페이스 설정**: 항상 임베딩 모델명을 네임스페이스로 사용하여 다른 모델과의 충돌을 방지하세요.
   ```python
   namespace=embedding.model  # 예: "text-embedding-ada-002"
   ```

2. **캐시 디렉토리 관리**: LocalFileStore 사용 시 적절한 디렉토리 구조를 유지하세요.
   ```python
   store = LocalFileStore("./cache/embeddings/")
   ```

3. **캐시 정리**: 필요 시 오래된 캐시를 정리하여 디스크 공간을 관리하세요.

4. **에러 처리**: 캐시 읽기/쓰기 실패에 대한 예외 처리를 고려하세요.

## 요약

`CacheBackedEmbeddings`는 임베딩 재계산을 방지하여 성능을 크게 향상시킵니다:

- **LocalFileStore**: 영구 캐싱이 필요한 프로덕션 환경에 적합
- **InMemoryByteStore**: 빠른 접근이 필요한 임시 작업에 적합
- 적절한 네임스페이스 사용으로 다른 모델과의 충돌 방지
- 첫 실행 후 30배 이상의 성능 향상 가능